### import

In [ ]:
# --- iPython Config --- #
from IPython import get_ipython
if 'IPython.extensions.autoreload' not in get_ipython().extension_manager.loaded:
    get_ipython().run_line_magic('load_ext', 'autoreload')
else:
    get_ipython().run_line_magic('reload_ext', 'autoreload')
%autoreload 2

# --- System and Path --- #
import os
import sys
REPO_PATH = os.path.abspath(os.path.join('..'))
if REPO_PATH not in sys.path:
    sys.path.append(REPO_PATH)
import warnings
warnings.filterwarnings("ignore")

# --- Data Manipulation --- #
import pandas as pd
import numpy as np
from tqdm import tqdm # Progress bar

# --- Financial Data --- #
import yfinance as yf

# --- Modules --- #
from src import *

# --- Visualization --- #
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# Read Data
df_transactions = pd.read_excel(REPO_PATH+'/data/private/main/'+'Transactions.xlsx')

# preprocessing

In [ ]:
def preprocess_securities(df):
    # Select Features
    df = df[['Date',
             'Position',
             'Ticker',
             'Executed Price (USD)',
             'Shares',
             ]]

    # Normalize Date
    df['Date'] = df['Date'].apply(lambda x: x.normalize())

    # New Features
    # Volume
    df['Position'] = df['Position'].apply(lambda x: 1 if x == "Buy" else -1)
    df['Volume'] = df['Shares'] * df['Position']
    df.drop(columns=['Shares', 'Position'], inplace=True)

    return df

df_transactions = preprocess_securities(df_transactions)

In [ ]:
def stock_split(df, ticker, split_date, split_ratio):
    # NVDA Stock Split 2024-06-10, 10:1

    # Volume Adjustment
    df.loc[
        (df["Ticker"] == ticker) & (df["Date"] < split_date),
        "Volume",
    ] = (df["Volume"] * split_ratio)

    # Price Adjustment
    df.loc[
        (df["Ticker"] == ticker) & (df["Date"] < split_date),
        "Executed Price (USD)",
    ] = (df["Executed Price (USD)"] / split_ratio)
    return df

df_transactions = stock_split(df_transactions, 'NVDA', '2024-06-10', 10)

# Portfolio

In [ ]:
df_portfolio = df_transactions.copy()

In [ ]:
df_portfolio = df_portfolio[~df_portfolio['Ticker'].isin(['GOOGL', 'MSFT', 'NVDA'])]

In [ ]:
def aggregate_intraday_to_daily(df):
    df['Date'] = df['Date'].dt.normalize()

    df = (
        df.groupby(["Ticker", "Date"])
        .apply(lambda x: pd.Series({
            "Executed Price (USD)": np.average(x["Executed Price (USD)"], weights=x["Volume"]),
            "Volume": x["Volume"].sum()
        }))
        .reset_index()
    )

    return df

df_portfolio = aggregate_intraday_to_daily(df_portfolio)

In [ ]:
def cumulative_volume(df):
    df["Cumulative Volume"] = df.groupby("Ticker")["Volume"].cumsum()

    # remove small values resulting from decimal point arithmetic operations
    least_significant_digit = 1e-6
    df["Cumulative Volume"] = df["Cumulative Volume"].apply(
        lambda x: 0 if abs(x) < least_significant_digit else x
    )

    df.drop(columns=["Volume"], inplace=True)
    return df

df_portfolio = cumulative_volume(df_portfolio)

In [ ]:
def average_cost_price(df):
    df["Average Cost Price (USD)"] = (
        df.groupby("Ticker")
        .apply(
            lambda x: (x["Executed Price (USD)"] * x["Volume"]).cumsum() / x["Cumulative Volume"]
        )
        .replace([np.inf, -np.inf], np.nan)
        .fillna(method="ffill")
        .reset_index(drop=True)
    )
    # df.drop(columns=["Executed Price (USD)"], inplace=True)
    return df

df_portfolio = average_cost_price(df_portfolio)

In [ ]:
df_portfolio

In [ ]:
def daily_basis(df):
    TODAY = pd.Timestamp.today()
    all_dates = pd.date_range(start=df["Date"].min(), end=TODAY)

    # Ensure all dates are present for each ticker
    # and forward-filling
    df = (
        df.set_index("Date")
        .groupby("Ticker")
        .apply(lambda x: x.reindex(all_dates))
        .drop(columns="Ticker")
        .reset_index()
        .rename(columns={"level_1": "Date"})
    )

    return df

# df_portfolio = daily_basis(df_portfolio)

## Market Price

In [ ]:
DataTerminal = DataTerminal()

In [ ]:
tickers = set(df_portfolio['Ticker'].unique())
# tickers.add("THB=X")
print(tickers)

df_yf = DataTerminal.fetch_data(tickers)

In [ ]:
def merge_yf_to_portfolio(df_portfolio, df_yf):
    # Select useful features
    df_yf = df_yf[['Ticker', 'Date', 'Adj Close']]

    # Apply daily basis function
    df_yf = daily_basis(df_yf)

    # Merge dataframes
    df_portfolio = df_portfolio.merge(df_yf, on=['Ticker', 'Date'], how='left')
    df_portfolio.rename(columns={'Adj Close': 'Market Price (USD)'}, inplace=True)

    # Forward fill missing market prices
    df_portfolio['Market Price (USD)'] = df_portfolio.groupby('Ticker')['Market Price (USD)'].ffill()

    return df_portfolio

# Apply the function
df_portfolio = merge_yf_to_portfolio(df_portfolio, df_yf)

## Calculation

In [ ]:
def portfolio_value(df):
    df["Asset Value (USD)"] = (
        df["Market Price (USD)"] * df["Cumulative Volume"]
    )

    portfolio_value = df.groupby("Date")["Asset Value (USD)"].sum()
    portfolio_value.name = "Portfolio Value (USD)"

    df = df.merge(
        portfolio_value, how="left", left_on="Date", right_on="Date"
    )
    return df

df_portfolio = portfolio_value(df_portfolio)

In [ ]:
def realized_pnl(df):
    # Realized PnL (USD) occurs when the security is sold

    for index, row in df.iterrows():
        if row['Volume'] < 0: # Sell
            df.loc[index, 'Asset Realized PnL (USD)'] = (
                abs(row['Volume']) * (row['Executed Price (USD)'] - row['Average Cost Price (USD)']) # Executed
            )
        else:
            df.loc[index, 'Asset Realized PnL (USD)'] = 0

    # Cumulative Realized PnL
    df['Asset CumRealized PnL (USD)'] = df.groupby('Ticker')['Asset Realized PnL (USD)'].cumsum()

    return df

df_portfolio = realized_pnl(df_portfolio)

In [ ]:
def unrealized_pnl(df):
    # Unrealized PnL (USD) occurs when the security is still hold

    df['Asset Unrealized PnL (USD)'] = abs(df['Cumulative Volume']) * (df['Market Price (USD)'] - df['Average Cost Price (USD)'])

    return df

df_portfolio = unrealized_pnl(df_portfolio)

In [ ]:
def portfolio_return(df):

    portfolio_return = df.groupby("Date")["Asset CumRealized PnL (USD)"].sum() + df.groupby("Date")["Asset Unrealized PnL (USD)"].sum()
    portfolio_return.name = "Portfolio Return (USD)"
    print(portfolio_return)
    df = df.merge(portfolio_return, how="left", left_on="Date", right_on="Date")

    return df

df_portfolio = portfolio_return(df_portfolio)

In [ ]:
df_portfolio.sort_values(by=['Date', 'Ticker'], inplace=False)

In [ ]:
def portfolio_return(df):

    portfolio_return = df.groupby("Date")["Asset CumReturn (%)"].sum()
    portfolio_return.name = "Portfolio CumReturn (%)"

    df = df.merge(
        portfolio_return, how="left", left_on="Date", right_on="Date"
    )

    return df

df_portfolio = portfolio_return(df_portfolio)

In [ ]:
df_portfolio.sort_values(by='Date')

# Visualization

In [ ]:
# Grouping and calculating the mean portfolio return by date
df = df_portfolio.groupby("Date")["Portfolio CumReturn (%)"].mean() # series

fig = go.Figure()
COLOR = str(np.where(df[-1] >= 0, "rgba(0, 255, 0, 0.2)", "rgba(255, 0, 0, 0.2)"))
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df,
        mode="lines",
        name="Portfolio Return (%)",
        fill="tozeroy",
        fillcolor=COLOR,
        line=dict(color=COLOR),
    )
)

fig.update_layout(
    title="Portfolio Cumulative Return (%)",
    xaxis_title="Date",
    yaxis_title="Return (%)",
    template="plotly_dark",
)

# Displaying the plot
fig.show()